In [1]:
from simulation_utils import build_data_by_year

# Get data for 2022-2023 season
df = build_data_by_year(2022)

# Get data for the 2023-2024 season
df_2023 = build_data_by_year(2023)

## Setting up Training Data

### Analyze Elo vs Outcome for Real Results

In [2]:
import pandas as pd
from scipy.stats import pearsonr
from sklearn.model_selection import ParameterGrid

from simulation_utils import DecayMethod, build_elo_between_seasons, get_elo_dict_from_df, process_fixture_results

# Define the parameter grid
param_grid = {
    'k': [40],
    'decay_half_life': [19],
    'club_value_adjustment_factor': [300],
    'decay_method': [DecayMethod.BASE_RATING],
}

# Initialize best parameters and best correlation
best_params = None
best_correlation = -1

# Iterate over all combinations of parameters
for params in ParameterGrid(param_grid):
    k = params['k']
    decay_half_life = params['decay_half_life']
    club_value_adjustment_factor = params['club_value_adjustment_factor']
    decay_method = params['decay_method']
    
    # Process the fixture results for the 2022-2023 season and the 2023-2024 season
    df, results = process_fixture_results(df, k, decay_half_life, club_value_adjustment_factor, decay_method, None)

    # Get adjusted Elo dict
    adjusted_elo = get_elo_dict_from_df(build_elo_between_seasons(df, df_2023, club_value_adjustment_factor))

    df_2023, results_2023 = process_fixture_results(df_2023, k, decay_half_life, club_value_adjustment_factor, decay_method, adjusted_elo)

    # Calculate Elo vs Outcome correlation
    data_2022 = df[["Home Elo", "Away Elo", "Home Outcome"]].copy()
    data_2022["Season"] = "2022-2023"
    data_2023 = df_2023[["Home Elo", "Away Elo", "Home Outcome"]].copy()
    data_2023["Season"] = "2023-2024"
    data_2022_2024 = pd.concat([data_2022, data_2023])

    data_2022_2024["Elo Difference"] = data_2022_2024["Home Elo"] - data_2022_2024["Away Elo"]

    # Convert 'Home Outcome' to categorical type with specified categories
    data_2022_2024['Home Outcome'] = pd.Categorical(data_2022_2024['Home Outcome'], categories=[3, 1, 0], ordered=True)

    # Calculate correlation
    correlation, p_value = pearsonr(data_2022_2024["Elo Difference"], data_2022_2024["Home Outcome"])
    
    # If the correlation is better than the best found so far, update best correlation and best parameters
    if correlation > best_correlation:
        best_correlation = correlation
        best_params = params

print("Best parameters:", best_params)
print("Best correlation:", best_correlation)

# Calculate everything again with the best parameters
k = best_params['k']
decay_half_life = best_params['decay_half_life']
club_value_adjustment_factor = best_params['club_value_adjustment_factor']
decay_method = best_params['decay_method']

# Process the fixture results for the 2022-2023 season and the 2023-2024 season
df, results = process_fixture_results(df, k, decay_half_life, club_value_adjustment_factor, decay_method, None)

# Get adjusted Elo dict
adjusted_elo = get_elo_dict_from_df(build_elo_between_seasons(df, df_2023, club_value_adjustment_factor))

df_2023, results_2023 = process_fixture_results(df_2023, k, decay_half_life, club_value_adjustment_factor, decay_method, adjusted_elo)

# Calculate Elo vs Outcome correlation
correlation_df_columns = ["Home Elo", "Away Elo", "Home Outcome", "Home", "Away", "Home Score", "Away Score", "Utc Date"]
data_2022 = df[correlation_df_columns].copy()
data_2022["Season"] = "2022-2023"
data_2023 = df_2023[correlation_df_columns].copy()
data_2023["Season"] = "2023-2024"
data_2022_2024 = pd.concat([data_2022, data_2023])

data_2022_2024["Match Info"] = data_2022_2024["Home"] + " (" + data_2022_2024["Home Score"].astype(str) + ") - " + data_2022_2024["Away"] + " (" + data_2022_2024["Away Score"].astype(str) + ") on " + data_2022_2024["Utc Date"].astype(str)

data_2022_2024["Elo Difference"] = data_2022_2024["Home Elo"] - data_2022_2024["Away Elo"]

# Convert 'Home Outcome' to categorical type with specified categories
data_2022_2024['Home Outcome'] = pd.Categorical(data_2022_2024['Home Outcome'], categories=[3, 1, 0], ordered=True)

# Calculate correlation
correlation, p_value = pearsonr(data_2022_2024["Elo Difference"], data_2022_2024["Home Outcome"])

# Convert 'Home Outcome' to string type
data_2022_2024['Home Outcome'] = data_2022_2024['Home Outcome'].astype(str)

results

Best parameters: {'club_value_adjustment_factor': 300, 'decay_half_life': 19, 'decay_method': <DecayMethod.BASE_RATING: 2>, 'k': 40}
Best correlation: 0.6259379489239572


,Home Outcome,Home Elo,Away Outcome,Away Elo,Total Outcome,Total Elo
Team,,,,,,
Manchester City FC,52,1696.353395,37,1649.704207,89,3346.057602
Arsenal FC,45,1587.103779,39,1574.604442,84,3161.708222
Manchester United FC,48,1608.169328,27,1587.107284,75,3195.276612
Newcastle United FC,39,1592.334070,32,1581.766867,71,3174.100937
Liverpool FC,44,1622.549865,23,1606.652121,67,3229.201987
Brighton & Hove Albion FC,34,1563.581317,28,1542.763820,62,3106.345137
Aston Villa FC,38,1592.770181,23,1576.988455,61,3169.758635
Tottenham Hotspur FC,37,1497.706569,23,1513.049616,60,3010.756184
Brentford FC,37,1583.204047,22,1559.521209,59,3142.725257


#### Plotting Elo vs Outcome

In [3]:
import plotly.express as px
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
fig = px.scatter(
    data_2022_2024, x="Elo Difference", y="Home Outcome", color="Season",
    hover_data=["Match Info"], title=f"Elo Difference vs Outcome - Correlation: {correlation:.2f}, P-value: {p_value:.2f}"
)
fig.show()

<Figure size 1000x600 with 0 Axes>

## Training

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler

# For simplicity, let"s predict the home outcome based on Elo ratings
x = df[["Home Elo", "Away Elo"]]
y = df["Home Outcome"]

# Split data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2)

# Standardize the features
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

# Define the parameter grid for Random Forest
# Current parameters: n_estimators=1000, max_depth=5, min_samples_split=5
param_grid_rf = {
    'n_estimators': [100, 200, 500, 1000],
    'max_depth': [None, 5, 10, 15, 20, 25],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Initialize the model
rf = RandomForestClassifier()

# Initialize GridSearchCV with 5-fold cross-validation
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid_rf, cv=5, n_jobs=-1, scoring='accuracy')

# Perform the grid search
grid_search.fit(x_train, y_train)

# Extract best parameters and model
print(f"Best parameters found: {grid_search.best_params_}")
print(f"Best cross-validation accuracy: {grid_search.best_score_}")

best_rf = grid_search.best_estimator_

# Cross-validate the model against 2023-2024 season
x_2023 = df_2023[["Home Elo", "Away Elo"]]
y_2023 = df_2023["Home Outcome"]

# Standardize the features
x_2023 = scaler.transform(x_2023)

# Validate the model
val_score_2023 = best_rf.score(x_2023, y_2023)
print(f"Validation accuracy on 2023-2024 data: {val_score_2023 * 100:.2f}%")

Fitting 5 folds for each of 432 candidates, totalling 2160 fits
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; tot

### Build Elo

In [10]:
elo_df = build_elo_between_seasons(df, df_2023, club_value_adjustment_factor)
elo_df

,Elo,Club Value,Normalized Club Value,Exponential Club Value,Normalized Exponential Club Value,Adjusted Elo
Team,,,,,,
Manchester City FC,1673.028801,1.320000e+09,1.000000,1.718282,1.000000,1973.028801
Arsenal FC,1580.854111,1.160000e+09,0.867159,1.380140,0.803209,1821.816900
Liverpool FC,1614.600993,9.419000e+08,0.686081,0.985917,0.573781,1786.735216
Manchester United FC,1597.638306,7.246500e+08,0.505708,0.658159,0.383033,1712.548282
Newcastle United FC,1587.050468,6.489500e+08,0.442858,0.557151,0.324249,1684.325103
Aston Villa FC,1584.879318,6.556000e+08,0.448379,0.565772,0.329266,1683.659138
Tottenham Hotspur FC,1505.378092,7.933000e+08,0.562705,0.755414,0.439634,1637.268172
Chelsea FC,1459.215136,9.460000e+08,0.689485,0.992689,0.577722,1632.531645
Brighton & Hove Albion FC,1553.172568,5.117500e+08,0.328947,0.389504,0.226682,1621.177239


## Test Against 2023-2024 Season

### Predict 2023-2024 Season

In [11]:
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
from functools import partial

from simulation_utils import simulate_and_get_results

# Get the model
model = best_rf

# Get adjusted Elo dict
adjusted_elo = get_elo_dict_from_df(elo_df)

# Number of simulations to run
num_simulations = 100

# Initialize a list to store results
seasons = []

# Create a partial function to pass the same arguments to each simulation
simulate_and_get_results_partial = partial(simulate_and_get_results, df=df_2023, elo=adjusted_elo, model=model, scaler=scaler, k=k, half_life=decay_half_life, decay_method=decay_method)

# Initialize a pool of workers
with ProcessPoolExecutor() as executor:
    seasons = list(tqdm(executor.map(simulate_and_get_results_partial, range(num_simulations)),  total=num_simulations, desc='Simulating', unit='season'))

Simulating: 100%|██████████| 100/100 [03:49<00:00,  2.29s/season]


### Analyze Results Compared to actual 2023-2024 Season

### Post Process Simulation Results

In [12]:
# Sort results based on total outcome
results = results.sort_values("Total Outcome", ascending=False)

# Get the place each team finished in the league
results["Place"] = range(1, len(results) + 1)

# Get 2023 season results
# Determine outcomes: 3 for win, 1 for draw, 0 for loss
df_2023["Home Outcome"] = 1
df_2023["Away Outcome"] = 1
df_2023.loc[df_2023["Home Score"] > df_2023["Away Score"], "Home Outcome"] = 3
df_2023.loc[df_2023["Home Score"] > df_2023["Away Score"], "Away Outcome"] = 0
df_2023.loc[df_2023["Away Score"] > df_2023["Home Score"], "Away Outcome"] = 3
df_2023.loc[df_2023["Away Score"] > df_2023["Home Score"], "Home Outcome"] = 0

home_results = df_2023.groupby("Home").agg({"Home Outcome": "sum"})
away_results = df_2023.groupby("Away").agg({"Away Outcome": "sum"})
results_2023 = home_results.join(away_results, how="outer").fillna(0)
results_2023["Total Outcome"] = results_2023["Home Outcome"] + results_2023["Away Outcome"]
results_2023 = results_2023.sort_values("Total Outcome", ascending=False)
results_2023["Place"] = range(1, len(results_2023) + 1)

# For each season, get the place each team finished in the league
for index, season_df in enumerate(seasons):
    season_df["Place"] = range(1, len(season_df) + 1)

# Get the average place each team finished in the league
average_results = pd.concat(seasons).groupby("Team").agg({"Place": "mean"}).sort_values("Place")

# Get a mapping of team names to a list places they finished in the league
team_place_mapping = {}
for team in average_results.index:
    team_place_mapping[team] = [season_df.loc[team, "Place"] for season_df in seasons]

# Get the total number of seasons simulated
total_seasons = len(seasons)

# Get a mapping of times each team won the league
team_win_mapping = {}
for team in average_results.index:
    team_win_mapping[team] = sum([season_df.loc[team, "Place"] == 1 for season_df in seasons]) / total_seasons

# Get a mapping of times each team finished in the top 4
team_top_4_mapping = {}
for team in average_results.index:
    team_top_4_mapping[team] = sum([season_df.loc[team, "Place"] <= 4 for season_df in seasons]) / total_seasons

# Get a mapping of times each team finished in the bottom 3
team_bottom_3_mapping = {}
for team in average_results.index:
    team_bottom_3_mapping[team] = sum([season_df.loc[team, "Place"] >= len(season_df) - 3 for season_df in seasons]) / total_seasons

# Build a dataframe with the average place, times won, times in top 4, and times in bottom 3
average_results["Win Premier League"] = [team_win_mapping[team] for team in average_results.index]
average_results["Top 4"] = [team_top_4_mapping[team] for team in average_results.index]
average_results["Bottom 3"] = [team_bottom_3_mapping[team] for team in average_results.index]

# Show the dataframe
average_results

,Place,Win Premier League,Top 4,Bottom 3
Team,,,,
Arsenal FC,2.83,0.28,0.82,0.00
Manchester City FC,2.86,0.27,0.82,0.00
Liverpool FC,3.83,0.13,0.65,0.00
Manchester United FC,4.41,0.13,0.55,0.00
Aston Villa FC,5.64,0.07,0.38,0.00
Newcastle United FC,5.88,0.09,0.30,0.00
Chelsea FC,7.93,0.02,0.08,0.00
Tottenham Hotspur FC,7.97,0.00,0.20,0.01
Brighton & Hove Albion FC,8.48,0.00,0.07,0.00
